In [1]:
!pip install langchain langchain-community langchain-groq langgraph sqlalchemy

Defaulting to user installation because normal site-packages is not writeable


## import the library's

In [2]:
import os
import sqlite3
from typing_extensions import TypedDict, Annotated

In [3]:
from langchain_groq import ChatGroq
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import START, StateGraph

## # ✅ Set Groq API key

In [ ]:
os.environ["GROQ_API_KEY"] = "apikey"

In [ ]:
# Store your API key
groq_api_key = "apikey"

## ✅ Connect to ARGO database

## create the llm

In [6]:
# ✅ Limit schema introspection to avoid giant prompts
db = SQLDatabase.from_uri("sqlite:///argo.db")

# ✅ Initialize Groq LLM (Mistral)
# Initialization of Groq LLM with a supported model
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="Gemma2-9b-It",   # use lowercase, as Groq model IDs are case-sensitive
    temperature=0
)
# -------------------------------


## check llm

In [7]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000023341B47FE0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023341C074D0>, model_name='Gemma2-9b-It', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
from langchain_groq import ChatGroq
from langchain_community.utilities import SQLDatabase
from langchain.agents import create_sql_agent
from langchain.agents.agent_types import AgentType

In [8]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002334190B0B0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002334190B0B0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002334190B0B0>),
 QuerySQLCheckerTool(description='Use this tool to 

In [9]:
system_message = """
You are an agent designed to interact with a SQLite database.

The database has 3 main tables:
- floats(id, platform_number)
- profiles(id, float_id, cycle_number, latitude, longitude, time)
- measurements(id, profile_id, pressure, temperature, salinity, time)

Relationships:
- A float can have many profiles (linked by profiles.float_id).
- A profile can have many measurements (linked by measurements.profile_id).

When the user asks about temperature, salinity, or pressure, you must query the
measurements table and JOIN with profiles if filtering by time, latitude, or longitude.

To filter by year or month, use SQLite functions like:
  strftime('%Y', measurements.time) and strftime('%m', measurements.time).

Do not invent tables or columns that do not exist.
""".format(
    dialect="SQLite",
    top_k=5,
)

In [10]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, prompt=system_message)

In [11]:
while True:
    # Take user input
    question = input("\nAsk a question about the database (or type 'exit' to quit): ")

    if question.lower() in ["exit", "quit", "q"]:
        print("Goodbye! 👋")
        break

    # Stream agent responses step by step
    for step in agent_executor.stream(
        {"messages": [{"role": "user", "content": question}]},
        stream_mode="values",
    ):
        step["messages"][-1].pretty_print()



Ask a question about the database (or type 'exit' to quit):  what is the min temp, pressure from 10/03/2020 to 20/03/2020


================================ Human Message =================================

what is the min temp, pressure from 10/03/2020 to 20/03/2020
================================== Ai Message ==================================
Tool Calls:
  sql_db_query (bcpxkx4e6)
 Call ID: bcpxkx4e6
  Args:
    query: SELECT MIN(temperature) AS min_temp, MIN(pressure) AS min_pressure FROM measurements WHERE strftime('%m', time) = '10' AND strftime('%Y', time) = '2020' AND strftime('%d', time) BETWEEN '03' AND '20'
================================= Tool Message =================================
Name: sql_db_query

[(None, None)]
================================== Ai Message ==================================

There are no temperature or pressure readings between 10/03/2020 and 20/03/2020.



Ask a question about the database (or type 'exit' to quit):  what is the min temp, pressure from 03/10/2020 to 03/20/2020


================================ Human Message =================================

what is the min temp, pressure from 03/10/2020 to 03/20/2020
================================== Ai Message ==================================
Tool Calls:
  sql_db_query (f8zfj9pz0)
 Call ID: f8zfj9pz0
  Args:
    query: SELECT MIN(temperature) AS min_temp, MIN(pressure) AS min_pressure FROM measurements WHERE strftime('%m', time) = '03' AND strftime('%Y', time) = '2020' AND strftime('%d', time) BETWEEN '10' AND '20'
================================= Tool Message =================================
Name: sql_db_query

[(2.552000045776367, 1.0)]
================================== Ai Message ==================================

The minimum temperature is 2.552 and the minimum pressure is 1.0.



Ask a question about the database (or type 'exit' to quit):  what is the min temp, pressure from 03/10/2020 to 03/20/2020 near indian ocean region


================================ Human Message =================================

what is the min temp, pressure from 03/10/2020 to 03/20/2020 near indian ocean region
================================== Ai Message ==================================
Tool Calls:
  sql_db_query (dbnhqncfk)
 Call ID: dbnhqncfk
  Args:
    query: SELECT MIN(temperature) AS min_temp, MIN(pressure) AS min_pressure FROM measurements JOIN profiles ON measurements.profile_id = profiles.id WHERE strftime('%m', measurements.time) = '03' AND strftime('%Y', measurements.time) = '2020' AND profiles.latitude BETWEEN -20 AND 10 AND profiles.longitude BETWEEN 50 AND 100
================================= Tool Message =================================
Name: sql_db_query

[(2.4489998817443848, 1.0)]
================================== Ai Message ==================================

The minimum temperature is 2.45 and the minimum pressure is 1.0.



Ask a question about the database (or type 'exit' to quit):  q


Goodbye! 👋
